In [ ]:
import pandas as pd
import csv
import re
import time

In [ ]:
import psycopg2
from sqlalchemy import create_engine, text

In [ ]:
db_user = "postgres"
db_password = ""
db_host = ""
db_name = "postgres"
db_schema = "public"

#Conectando ao POstgreSQL para retornar a modelagem de dados.
connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}"

# Create an engine instance
engine = create_engine(connection_string)

In [ ]:

# Consulta SQL para testar a conexão
test_query = text('SELECT count(*) from "public"."CNAE"')
with engine.connect() as connection:
    result = connection.execute(test_query).scalar()
    print(f"Test query result: {result}")


In [ ]:
# Carregar o arquivo CSV
df = pd.read_csv('gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/cnpjnl2sql/queries/queries.csv')


In [ ]:
# Adicionar colunas para resultados e tempo de execução
df['qtd_postgres'] = None
df['execution_time'] = None

# Executar consultas e armazenar resultados
for index, row in df.iterrows():
    query = row['postgres_query']
    print(f"Executing query on line {index + 1}: {query}")
    try:
        start_time = time.time()
        with engine.connect() as connection:
            result = connection.execute(text(query)).scalar()
        execution_time = time.time() - start_time
        df.at[index, 'qtd_postgres'] = result
        df.at[index, 'execution_time'] = execution_time
        print(f"Result: {result}, Execution Time: {execution_time} seconds")
    except Exception as e:
        df.at[index, 'qtd_postgres'] = str(e)
        df.at[index, 'execution_time'] = None
        print(f"Error on line {index + 1}: {e}")

# Salvar os resultados em um novo arquivo CSV
output_csv_path = "queries_results_postgres.csv"
df.to_csv(output_csv_path, index=False)

# Fechar o mecanismo de conexão (opcional, mas recomendado)
engine.dispose()

In [ ]:
df_queries.to_csv('gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/cnpjnl2sql/queries/queries_results_postgresql.csv', index=False)